# CSC555 Final Project - Christopher Elchik, Zack Haugan, Ethan Telep

Conducts simulations of information flow in different social network scenarios.

Following code is inspired by the given tutorials by Networkx and Mesa documentation, linked below:

Networkx: https://networkx.org/documentation/stable/tutorial.html
Mesa: https://mesa.readthedocs.io/latest/tutorials/intro_tutorial.html

Restart/run all cells to confirm outputs.

In [48]:
# Import dependencies
import networkx as nx
import matplotlib.pyplot as plt
from mesa import Agent
from mesa import Model
from mesa.time import RandomActivation # TODO: figure out how agents will be activated
import random
import numpy as np

# Should be 2.3.4
import mesa
print(mesa.__version__)

# define constants/global variables
RELATIVE_DATA_PATH = 'data/socialmediadata-social-circles-from-facebook/original/facebook/' # start of filepath to data files
EDGELIST_3980 = '3980.edges' # edge list representation with 292 edges
EDGELIST_3437 = '3437.edges' # edge list representation with 9626 edges
EDGELIST_686 = '686.edges' # edge list representation with 
EDGELIST_1912 = '1912.edges' # edge list representation with 
agents = {} # dictionary storage of agents since Mesa deprecated this for some reason

# load in dataset as a directed graph
digraph = nx.DiGraph()
# G = nx.read_edgelist(f'{RELATIVE_DATA_PATH}{EDGELIST_3980}', create_using=digraph)
G3437 = nx.read_edgelist(f'{RELATIVE_DATA_PATH}{EDGELIST_3437}', create_using=nx.DiGraph())
G = nx.read_edgelist(f'{RELATIVE_DATA_PATH}{EDGELIST_686}', create_using=nx.DiGraph())

# show connectivity
# nx.betweenness_centrality()
# print(f'Graph 3980 Connectivity: {nx.average_node_connectivity(G)}')
# print(f'Graph 3980 Clustering: {nx.average_clustering(G)}')
# print(f'Graph 3437 Connectivity: {nx.average_node_connectivity(G3437)}')
# print(f'Graph 3437 Clustering: {nx.average_clustering(G3437)}')

2.3.4


In [49]:

# Define agents (nodes on graph)
class TrendAgent(Agent):
    """
    Agent to represent a node on the social graph. Uses Mesa Framework's agent structure.
    
    Has an apathy rate, engagement value, and susceptibility value from all neighbors.
    """

    def __init__(self, unique_id:str, model:Model, apathy_rate:float, initial_engagement:float, suceptibility_dist:tuple):
        """
        Sets up an agent in the simulated trend spreading model.

        Args:
            unique_id: ID of agent
            model: model agent belongs to
            apathy_rate: rate at which agent loses interest in trend
            initial_engagement: initial value of agent's interest in given trend
        """
        super().__init__(unique_id, model)
        self.unique_id = unique_id
        self.apathy_rate = apathy_rate
        self.engagement = initial_engagement
        self.neighborsSusceptibility = {agent:random.uniform(suceptibility_dist[0], suceptibility_dist[1]) for agent in model.G.predecessors(unique_id)} # an agent will map the neighbors from incoming edges to the suceptibility value for that edge
                                                            # Note: neighbors are inward edges in networkx digraphs, representing a 'followed by' relationship
        # agents[unique_id] = self # add to agents dict
        self.model = model

        self.counter = 0
        self.peakEngagamentStep = 0
        self.peakEngagamentValue = initial_engagement

        # all 'update' variables
        self.updatedEngagement = initial_engagement
        self.updatedNeighborsSusceptibility = self.neighborsSusceptibility.copy()

    def step(self):
        """
        Step function for Mesa framework. Gets called with each time step.
        """
        
        # initialize all new values
        self.updatedEngagement = self.engagement
        self.updatedNeighborsSusceptibility = self.neighborsSusceptibility.copy()

        # aggregate influence by all outward edges
        for neighbor, susceptibility in self.neighborsSusceptibility.items():
            
            # get neighboring agent obj
            nbr_agent = self.model.agentDict[neighbor]

            # probability of increasing engagement = the susceptibility of the incoming edge
            if random.uniform(0,1) < susceptibility:
                self.updatedEngagement += susceptibility * nbr_agent.getEngagement()
            else:
                # decrease engagement slightly
                self.updatedEngagement -= (self.apathy_rate) * 0.1

            # keep engagement between 0 and 1
            self.updatedEngagement = min(1, max(0, self.updatedEngagement))

            # scale susceptibility by apathy every step
            self.updatedNeighborsSusceptibility[neighbor] = self.neighborsSusceptibility[neighbor] * (1-self.apathy_rate)
        
        self.counter += 1
        
        # see if new engagement is peak engagement
        if (self.updatedEngagement > self.peakEngagamentValue):
            self.peakEngagamentValue = self.updatedEngagement
            self.peakEngagamentStep = self.counter

    def updateValues(self):
        self.engagement = self.updatedEngagement
        self.neighborsSusceptibility = self.updatedNeighborsSusceptibility.copy()

    def getEngagement(self):
        return self.engagement

    def getGraphDegree(self):
        return self.model.G.degree(self.unique_id)

    def getPeakEngagementStep(self):
        return self.peakEngagamentStep

        

In [50]:
# Define model/environment for agents
class TrendModel(Model):
    """
    Model containing social graph. Uses Mesa Framework's model structure.

    Has a networkx digraph.
    """
    def __init__(self, G:nx.DiGraph, initial_engagement:float, suceptibility_dist:tuple, apathy_rate_upper_bound:float, betweeness_mapping:dict):
        """
        Sets up a model to simulate trend spreading.

        Args:
            G: pre-made graph, set up by networkx (DiGraph object)
        """
        super().__init__()
        self.G = G
        self.counter = 0
        self.schedule = RandomActivation(self)
        self.engagement_rate = []
        self.betweeness_mapping = betweeness_mapping
        self.agentDict = {}



        # create/assign all agents/nodes
        for i, node in enumerate(self.G.nodes):
            agent_apathy_rate = random.uniform(0, apathy_rate_upper_bound)
            agent_initial_engagement = initial_engagement if i == 0 else 0
            agent = TrendAgent(unique_id=node, model=self, apathy_rate=agent_apathy_rate, initial_engagement=agent_initial_engagement, suceptibility_dist=suceptibility_dist)
            self.agentDict[node] = agent
            self.schedule.add(agent)

        self.engagement_data = [np.average([agent.getEngagement() for agent in self.schedule.agents])] # collect total average engagement data
    
    def step(self):
        """
        Step function for Mesa Framework. Occurs each timestep; calls all agent step functions
        """
        self.engagement_data.append(np.average([agent.getEngagement() for agent in self.agentDict.values()]))
        self.engagement_rate.append(self.engagement_data[-1] - self.engagement_data[-2])
        # loop through all nodes and call updatevalues()
        for agent in self.agentDict.values():
            agent.step()
        for agent in self.agentDict.values():
            agent.updateValues()
        self.counter += 1


    
    def runForSteps(self, numSteps=200):
        for i in range(numSteps):
            self.step()
    
    def getEngagementData(self):
        return self.engagement_data
    
    def getEngagementRate(self):
        return self.engagement_rate




# Metrics for keeping track
- Plot Engagement for different node degrees
- derivative of engagement graph to show "speed"


# Configurations to run
- Graph structures: 3980 (292 edges), 698 (540 edges), 686 (3312 edges), 1912 (60050 edges)
- Initial engagement of trendsetter: [0.25, 0.5, 0.75, 1]
- Suceptibility distribution: (0-0.5), (0.25-0.75), (0.5-1)
- Apathy Rate: between 0 and [0.1, 0.25, 0.5]



In [54]:
# Code for generating average total engagement graphs

import matplotlib.pyplot as plt
import os
import itertools

# Create a directory to save the plots
OUTPUT_DIR = "output_graphs"
os.makedirs(OUTPUT_DIR, exist_ok=True)

graph_structures =  [EDGELIST_3980, EDGELIST_3437, EDGELIST_686, EDGELIST_1912]
initial_engagements = [0.25, 0.5, 0.75, 1]
suceptibility_distribution = [(0,0.5), (0.25,0.75), (0.5,1)]
apathy_rate = [0.1, 0.25, 0.5]

# Create a directory to save the plots
OUTPUT_DIR = "output_graphs"
os.makedirs(OUTPUT_DIR, exist_ok=True)

# Iterate over graph structures
for graph_structure in graph_structures:
    G = nx.read_edgelist(f'{RELATIVE_DATA_PATH}{graph_structure}', create_using=nx.DiGraph())
    betweenness = nx.betweenness_centrality(G, normalized=True)
    
    # Vary one variable while keeping the others constant
    for varying_variable, variable_values in {
        "initial_engagement": initial_engagements,
        "susceptibility_distribution": suceptibility_distribution,
        "apathy_rate": apathy_rate
    }.items():
        # Identify the constant variables
        constant_variables = {
            "initial_engagement": initial_engagements,
            "susceptibility_distribution": suceptibility_distribution,
            "apathy_rate": apathy_rate,
        }
        del constant_variables[varying_variable]  # Remove the varying variable

        # Generate all combinations of constant variables
        constant_combinations = list(itertools.product(*constant_variables.values()))
        
        for constants in constant_combinations:
            
            # store fixed values for constants
            if varying_variable == "initial_engagement":
                fixed_initial_engagement = "varied"
                fixed_susceptibility = constants[0]
                fixed_apathy_rate = constants[1]
            elif varying_variable == "susceptibility_distribution":
                fixed_initial_engagement = constants[0]
                fixed_susceptibility = "varied"
                fixed_apathy_rate = constants[1]
            elif varying_variable == "apathy_rate":
                fixed_initial_engagement = constants[0]
                fixed_susceptibility = constants[1]
                fixed_apathy_rate = "varied"
                
            results = {}  # Store engagement data for each varying value
            rates = {} # stores max and min rates
            # Run simulations for the varying variable
            for value in variable_values:
                # Create model with the appropriate varying and fixed values
                if varying_variable == "initial_engagement":
                    model = TrendModel(G, value, fixed_susceptibility, fixed_apathy_rate, betweenness)
                elif varying_variable == "susceptibility_distribution":
                    model = TrendModel(G, fixed_initial_engagement, value, fixed_apathy_rate, betweenness)
                elif varying_variable == "apathy_rate":
                    model = TrendModel(G, fixed_initial_engagement, fixed_susceptibility, value, betweenness)
                
                # Run the model and collect engagement data
                model.runForSteps(200)
                engagement_data = model.getEngagementData()
                engagement_rate = model.getEngagementRate()
                max_rate, min_rate = max(engagement_rate), min(engagement_rate)
                results[value] = engagement_data  # Store results for plotting
                rates[value] = (round(max_rate, 3), round(min_rate, 3))
            
            # Generate the plot
            plt.figure(figsize=(10, 6))
            for value, engagement_data in results.items():
                plt.plot(engagement_data, label=f"{varying_variable} = {value} ({rates[value][0]}, {rates[value][1]})")
            
            # Generate a descriptive title for the plot
            title = (f"Graph: {graph_structure}\n"
                     f"initial_engagement={fixed_initial_engagement}, "
                     f"susceptibility_dist={fixed_susceptibility}, "
                     f"apathy_rate={fixed_apathy_rate}")
            plt.title(title)
            plt.xlabel("Steps")
            plt.ylabel("Average Agent Engagement")
            plt.ylim((0,1))
            plt.legend()
            plt.grid(True)
            
            # Save the plot with a descriptive filename
            filename = (f"{graph_structure}_"
                        f"initial_{fixed_initial_engagement}_"
                        f"susceptibility_{fixed_susceptibility}_"
                        f"apathy_{fixed_apathy_rate}.png").replace(" ", "_")
            filepath = os.path.join(OUTPUT_DIR, filename)
            plt.savefig(filepath)
            print(f"{filename} Generated")
            plt.close()



1912.edges_initial_varied_susceptibility_(0,_0.5)_apathy_0.1.png Generated
1912.edges_initial_varied_susceptibility_(0,_0.5)_apathy_0.25.png Generated
1912.edges_initial_varied_susceptibility_(0,_0.5)_apathy_0.5.png Generated
1912.edges_initial_varied_susceptibility_(0.25,_0.75)_apathy_0.1.png Generated
1912.edges_initial_varied_susceptibility_(0.25,_0.75)_apathy_0.25.png Generated
1912.edges_initial_varied_susceptibility_(0.25,_0.75)_apathy_0.5.png Generated
1912.edges_initial_varied_susceptibility_(0.5,_1)_apathy_0.1.png Generated
1912.edges_initial_varied_susceptibility_(0.5,_1)_apathy_0.25.png Generated
1912.edges_initial_varied_susceptibility_(0.5,_1)_apathy_0.5.png Generated
1912.edges_initial_0.25_susceptibility_varied_apathy_0.1.png Generated
1912.edges_initial_0.25_susceptibility_varied_apathy_0.25.png Generated
1912.edges_initial_0.25_susceptibility_varied_apathy_0.5.png Generated
1912.edges_initial_0.5_susceptibility_varied_apathy_0.1.png Generated
1912.edges_initial_0.5_sus

In [55]:
graph_structures =  [EDGELIST_3980, EDGELIST_3437, EDGELIST_686, EDGELIST_1912]

for graph_structure in graph_structures:
    G = nx.read_edgelist(f'{RELATIVE_DATA_PATH}{graph_structure}', create_using=nx.DiGraph())
    print(f"Network: {graph_structure}")
    print(f"Nodes: {len(G.nodes)}")
    print(f"Edges: {len(G.edges)}")
    print(f"Average Degree: {sum(dict(G.degree()).values()) / len(G.nodes):.2f}")
    print(f"Average Clustering Coefficient: {nx.average_clustering(G):.4f}")
    print(f"Transitivity (Global Clustering Coefficient): {nx.transitivity(G):.4f}")




Network: 3980.edges
Nodes: 52
Edges: 292
Average Degree: 11.23
Average Clustering Coefficient: 0.4617
Transitivity (Global Clustering Coefficient): 0.4500
Network: 3437.edges
Nodes: 534
Edges: 9626
Average Degree: 36.05
Average Clustering Coefficient: 0.5437
Transitivity (Global Clustering Coefficient): 0.4489
Network: 686.edges
Nodes: 168
Edges: 3312
Average Degree: 39.43
Average Clustering Coefficient: 0.5338
Transitivity (Global Clustering Coefficient): 0.4536
Network: 1912.edges
Nodes: 747
Edges: 60050
Average Degree: 160.78
Average Clustering Coefficient: 0.6354
Transitivity (Global Clustering Coefficient): 0.7000
